In [3]:
import numpy as np
import pandas as pd

def topsis(decision_matrix, weights, benefit_criteria):
    decision_matrix = np.array(decision_matrix, dtype=float)
    weights = np.array(weights, dtype=float)
    benefit_criteria = np.array(benefit_criteria, dtype=int)  # Convert to numpy array

    # Step 1: Normalize the Decision Matrix
    norm_matrix = decision_matrix / np.sqrt((decision_matrix**2).sum(axis=0))

    # Step 2: Weighted Normalized Decision Matrix
    weighted_matrix = norm_matrix * weights

    # Step 3: Identify Ideal (Best) and Negative-Ideal (Worst) Solutions
    ideal_best = np.max(weighted_matrix, axis=0) * benefit_criteria + np.min(weighted_matrix, axis=0) * (1 - benefit_criteria)
    ideal_worst = np.min(weighted_matrix, axis=0) * benefit_criteria + np.max(weighted_matrix, axis=0) * (1 - benefit_criteria)

    # Step 4: Compute the Euclidean Distance from Ideal and Negative-Ideal Solutions
    dist_best = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))

    # Step 5: Compute the Relative Closeness Score
    score = dist_worst / (dist_best + dist_worst)

    return score

# Define the decision matrix
decision_matrix = [
    [0.85, 50, 400, 5, 0.78],  # BERT
    [0.88, 40, 500, 6, 0.82],  # RoBERTa
    [0.82, 200, 100, 1, 0.76], # SBERT
    [0.90, 5, 3500, 50, 0.85]  # GPT-3.5
]

weights = [0.3, 0.2, 0.2, 0.2, 0.1]  # Adjust based on preference

benefit_criteria = np.array([1, 1, 0, 0, 1])  # Convert to numpy array

scores = topsis(decision_matrix, weights, benefit_criteria)

model_names = ["BERT", "RoBERTa", "SBERT", "GPT-3.5"]
ranked_models = sorted(zip(model_names, scores), key=lambda x: x[1], reverse=True)

result_df = pd.DataFrame(ranked_models, columns=["Model", "TOPSIS Score"])
result_df["Rank"] = result_df["TOPSIS Score"].rank(ascending=False, method="dense").astype(int)
print(result_df)


     Model  TOPSIS Score  Rank
0    SBERT      0.956462     1
1     BERT      0.635335     2
2  RoBERTa      0.611742     3
3  GPT-3.5      0.043538     4
